### Libraries

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
import datetime

### HTTP Requests

In [2]:
# big ecommerce
web_sephora = 'https://www.sephora.co.id/search?q=moisturizer' #req can


In [3]:
response = requests.get(web_sephora)

In [4]:
response.status_code

200

### Soup Object

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
soup

<!DOCTYPE html>

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
<head>
<title>Pencarian | Sephora Indonesia</title><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1" data-n-head="ssr" name="viewport"/><meta content="SEPHORA" data-n-head="ssr" name="author"/><meta content="#000" data-n-head="ssr" name="theme-color"/><meta content="https://static-reg.lximg.com/assets/sephora_og_image-34f1169559fdb47662226655a01e2f90dabe8622664f82b7f9b4ed5907237b3d.jpg" data-n-head="ssr" id="og-image" name="og:image" property="og:image"/><meta content="website" data-n-head="ssr" id="og-type" name="og:type" property="og:type"/><meta content="Sephora" data-n-head="ssr" id="og-site-name" property="og:site_name"/><meta content="false" data-n-head="ssr" property="al:web:should_fallback"/><meta charset="utf-8" data-hid="charset" data-n-head="ssr"/><meta content="yes" data-hid="mobile-web-app-capable" data-n-head="ssr" name="m

In [7]:
results = soup.find_all('div', {'class': 'products-card-container'})

In [8]:
len(results)

36

In [9]:
results[0]

<div class="products-card-container" data-v-2034eb20=""><div class="product-card" data-list="search" data-position="1" data-product-brand="Bobbi Brown" data-product-id="17518" data-product-name="Vitamin Enriched Face Base - Moisturizer and Primer" data-v-893a5fe2=""><div class="product-card-details" data-v-893a5fe2=""><div class="product-card-ad" data-v-893a5fe2=""><!-- --></div> <!-- --> <a aria-label="/products/bobbi-brown-vitamin-enriched-face-base-moisturizer-and-primer/v/7ml" class="product-card-image-link" data-v-893a5fe2="" href="/products/bobbi-brown-vitamin-enriched-face-base-moisturizer-and-primer/v/7ml"><img alt="Vitamin Enriched Face Base - Moisturizer and Primer" class="product-card-image" loading="lazy" src="https://image-optimizer-id.production.sephora-asia.net/eyJlZGl0cyI6eyJyZXNpemUiOnt9fX0=/images/product_images/default_1_Product_716170269443-Bobbi-Brown-Vitamin-Enriched-Face-Base-Face-Primer-7ml_8b82231c699f6994f5310ef4c7540690221f546f_1627052192.png"/></a> <!-- --> 

### Sephora Scraping

In [13]:
# Link              V
# Brand
# Product Name      V
# Price             V
# Rating            V
# Sold Count        V
# Sizes

In [15]:
# origin

# get all of the info of the ecommerce
product_cards = results[0].find_all('div', {'class':'product-card'})

# Extract data-product-brand and data-product-name
if product_cards:
    for card in product_cards:
        product_name = card.get('data-product-name')
        product_brand = card.get('data-product-brand')
        # Extract sell price
        sell_price = card.find('span', class_='sell-price').get_text(strip=True) if card.find('span', class_='sell-price') else 'N/A'
        
        # Extract reviews count
        reviews_count = card.find('span', class_='reviews-count').get_text(strip=True) if card.find('span', class_='reviews-count') else 'N/A'
        
        # Extract rating
        rating = card.find('div', class_='stars')['style'] if card.find('div', class_='stars') else 'N/A'
        highlighted_percentage = 'N/A'
        
        if rating != 'N/A':
            # Use regex to find the highlighted percentage value
            match = re.search(r'--highlighted-percentage:(\d+%)', rating)
            if match:
                highlighted_percentage = match.group(1)  # Get the percentage value
        
        print(f'Product Name: {product_name}, Brand: {product_brand}, Sell Price: {sell_price}, Reviews Count: {reviews_count}, Highlighted Percentage: {highlighted_percentage}')
else:
    print('No product card found.')

No product card found.


In [14]:
def get_skincare_data(results):
    # Initialize a list to hold the product data
    product_data = []

    # Loop through each index in results
    for index in range(len(results)):
        # Get all of the info of the ecommerce
        product_cards = results[index].find_all('div', {'class': 'product-card'})

        # Extract data-product-brand and data-product-name 
        for card in product_cards:
            product_name = card.get('data-product-name')
            product_brand = card.get('data-product-brand')
            
            # Extract sell price
            sell_price_text = card.find('span', class_='sell-price').get_text(strip=True) if card.find('span', class_='sell-price') else 'N/A'
            sell_price = int(sell_price_text.replace('Rp. ', '').replace('.', '').strip()) if 'Rp.' in sell_price_text else 'N/A'
            
            # Extract reviews count
            reviews_count_text = card.find('span', class_='reviews-count').get_text(strip=True) if card.find('span', class_='reviews-count') else 'N/A'
            reviews_count = int(reviews_count_text.replace('(', '').replace(')', '').strip()) if '(' in reviews_count_text else 'N/A'
            
            # Extract rating
            rating = card.find('div', class_='stars')['style'] if card.find('div', class_='stars') else 'N/A'
            highlighted_percentage = 'N/A'
            
            if rating != 'N/A':
                # Use regex to find the highlighted percentage value
                match = re.search(r'--highlighted-percentage:(\d+)%', rating)
                if match:
                    highlighted_percentage = int(match.group(1))  # Convert to integer
            
            # Append the extracted data to the list
            product_data.append({
                'brand': product_brand,
                'product_name': product_name,
                'price': sell_price,
                'reviews_count': reviews_count,
                'rating_percentage': highlighted_percentage
            })

    # Create a DataFrame from the list of product data
    df = pd.DataFrame(product_data)
    
    return df

# Call the function and print the DataFrame
print(get_skincare_data(results))


Empty DataFrame
Columns: []
Index: []


In [ ]:
# refactoring functions v1.0
def get_sephora_data(ti, page_no, skincare_product):
    if page_no > 1:
        page_num = 1
        while page_num <= page_no:
            url = f'https://www.sephora.co.id/search?page={page_num}&q={skincare_product}'
            headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
            
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                results = soup.find_all('div', {'class': 'products-card-container'})
            except Exception as e:
                raise Exception(f"Failed to fetch or parse page: {e}")

            product_data = []

            for container in results:
                product_cards = container.find_all('div', {'class': 'product-card'})
                for card in product_cards:
                    product_name = card.get('data-product-name')
                    product_brand = card.get('data-product-brand')

                    # Price
                    sell_price_text = card.find('span', class_='sell-price')
                    if sell_price_text:
                        sell_price_text = sell_price_text.get_text(strip=True)
                        # Remove Rp, dots, spaces
                        cleaned = re.sub(r'[^\d]', '', sell_price_text)  # Keeps only digits
                        sell_price = int(cleaned) if cleaned.isdigit() else None
                    else:
                        sell_price = None

                    # Reviews count
                    reviews_elem = card.find('span', class_='reviews-count')
                    reviews_text = reviews_elem.get_text(strip=True) if reviews_elem else 'N/A'
                    reviews_count = int(reviews_text.replace('(', '').replace(')', '')) \
                        if '(' in reviews_text else None

                    # Extract rating percentage from style attribute
                    stars_elem = card.find('div', class_='stars')
                    rating_style = stars_elem['style'] if stars_elem else None
                    rating_percentage = None  # Default to None

                    if rating_style:
                        # Use regex to extract the percentage number
                        match = re.search(r'--highlighted-percentage:\s*([0-9.]+)%', rating_style)
                        if match:
                            try:
                                # Convert to float first, then int (e.g., "88.000..." → 88)
                                rating_percentage = int(float(match.group(1)))
                            except ValueError:
                                rating_percentage = None  # In case conversion fails

                    product_data.append({
                        'brand': product_brand,
                        'product_name': product_name,
                        'price': sell_price,
                        'reviews_count': reviews_count,
                        'rating_percentage': rating_percentage
                    })

            df = pd.DataFrame(product_data)
            ti.xcom_push(key='skincare_data', value=df.to_dict('records'))

            page_num += 1

            #politeness delay
            time.sleep(1)
        
    else:
        url = f'https://www.sephora.co.id/search?q={skincare_product}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            results = soup.find_all('div', {'class': 'products-card-container'})
        except Exception as e:
            raise Exception(f"Failed to fetch or parse page: {e}")

        product_data = []

        for container in results:
            product_cards = container.find_all('div', {'class': 'product-card'})
            for card in product_cards:
                product_name = card.get('data-product-name')
                product_brand = card.get('data-product-brand')

                # Price
                sell_price_text = card.find('span', class_='sell-price')
                if sell_price_text:
                    sell_price_text = sell_price_text.get_text(strip=True)
                    # Remove Rp, dots, spaces
                    cleaned = re.sub(r'[^\d]', '', sell_price_text)  # Keeps only digits
                    sell_price = int(cleaned) if cleaned.isdigit() else None
                else:
                    sell_price = None

                # Reviews count
                reviews_elem = card.find('span', class_='reviews-count')
                reviews_text = reviews_elem.get_text(strip=True) if reviews_elem else 'N/A'
                reviews_count = int(reviews_text.replace('(', '').replace(')', '')) \
                    if '(' in reviews_text else None

                # Extract rating percentage from style attribute
                stars_elem = card.find('div', class_='stars')
                rating_style = stars_elem['style'] if stars_elem else None
                rating_percentage = None  # Default to None

                if rating_style:
                    # Use regex to extract the percentage number
                    match = re.search(r'--highlighted-percentage:\s*([0-9.]+)%', rating_style)
                    if match:
                        try:
                            # Convert to float first, then int (e.g., "88.000..." → 88)
                            rating_percentage = int(float(match.group(1)))
                        except ValueError:
                            rating_percentage = None  # In case conversion fails

                product_data.append({
                    'brand': product_brand,
                    'product_name': product_name,
                    'price': sell_price,
                    'reviews_count': reviews_count,
                    'rating_percentage': rating_percentage
                })

        df = pd.DataFrame(product_data)
        ti.xcom_push(key='skincare_data', value=df.to_dict('records'))


In [ ]:
# Fixing refactoring for sephora v2.0
def extract_product_data_from_sephora(card):
    """
    Extract product data from sephora card
    """
    product_name = card.get('data-product-name')
    product_brand = card.get('data-product-brand')

    # Price Extraction
    sell_price_text = card.find('span', class_='sell-price')
    if sell_price_text:
        sell_price_text = sell_price_text.get_text(strip=True)
        # Remove Rp, dots, spaces
        cleaned = re.sub(r'[^\d]', '', sell_price_text)  # Keeps only digits
        sell_price = int(cleaned) if cleaned.isdigit() else None
    else:
        sell_price = None

    # Reviews count
    reviews_elem = card.find('span', class_='reviews-count')
    reviews_text = reviews_elem.get_text(strip=True) if reviews_elem else 'N/A'
    reviews_count = int(reviews_text.replace('(', '').replace(')', '')) \
        if '(' in reviews_text else None
    
    # Extract rating percentage from style attribute
    stars_elem = card.find('div', class_='stars')
    rating_style = stars_elem['style'] if stars_elem else None
    rating_percentage = None  # Default to None

    if rating_style:
        # Use regex to extract the percentage number
        match = re.search(r'--highlighted-percentage:\s*([0-9.]+)%', rating_style)
        if match:
            try:
                # Convert to float first, then int (e.g., "88.000..." → 88)
                rating_percentage = int(float(match.group(1)))
            except ValueError:
                rating_percentage = None  # In case conversion fails
    
    return {
    'brand': product_brand,
    'product_name': product_name,
    'price': sell_price,
    'reviews_count': reviews_count,
    'rating_percentage': rating_percentage
    }


def get_sephora_data(ti, page_no, product_keyword):
    """
    Fetch skincare product data from Sephora Indonesia across multiple pages.
    Pushes result to Airflow's XCom.
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    all_product_data = []

    # determine page range
    start_page = 1
    end_page = max(page_no, 1)

    for page_num in range(start_page, end_page + 1):
        url = f'https://www.sephora.co.id/search?page={page_num}&q={product_keyword}'
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            results = soup.find_all('div', {'class': 'products-card-container'})

            for result in results:
                cards = result.find_all('div', class_='product-card')
                for card in cards:
                    product_data = extract_product_data_from_sephora(card)
                    all_product_data.append(product_data)

        except requests.RequestException as e:
            raise Exception(f"Failed to fetch page {page_num}: {e}")
        except Exception as e:
            raise Exception(f"Failed to parse page {page_num}: {e}")
        
        #Politeness Delay
        time.sleep(1)

    # Convert to DataFrame and push to XCom
    df = pd.DataFrame(all_product_data)
    ti.xcom_push(key='skincare_data', value=df.to_dict('records'))

SyntaxError: incomplete input (1483888335.py, line 4)

### BLP Beauty scrape

In [2]:
# big ecommerce HTTP Requests
web_skincare = 'https://blpbeauty.com/id/search?type=product&q=moisturizer' #req can
response = requests.get(web_skincare)
response.status_code

NameError: name 'requests' is not defined

In [ ]:
# Soup Object
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html dir="ltr" lang="id">
<head>
<!-- Blockshop theme - Version 12.1.1 by Troop Themes - https://troopthemes.com/ -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/><meta content="BLP Beauty Official Website. Enjoy the complete shades of BLP Beauty products and get free shipping" name="description"/><title>
      Pencarian: 39 hasil ditemukan untuk "moisturizer"
 – BLP Beauty</title><link href="https://blpbeauty.com/id/search?q=moisturizer&amp;type=product" rel="canonical"/><link href="//blpbeauty.com/cdn/shop/files/BLP_Logo_Short.png?v=1664966622&amp;width=32" rel="shortcut icon" type="image/png"/>
<link href="//blpbeauty.com/cdn/shop/files/BLP_Logo_Short.png?v=1664966622&amp;width=180" rel="apple-touch-icon"/><link crossorigin="" href="https://cdn.shopify.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.shopifycdn.com" rel="precon

In [ ]:
# Results
results = soup.find_all('div', {'class': 'product-card--root'})
# len(results)
results

[<div class="product-card--root" data-aspect-ratio="natural" data-container="block" data-handle="makeup-fixer" data-product-item="" data-text-layout="center" data-transition-item="template--18771463340206__main">
 <a class="product-card--image-wrapper" href="/id/products/makeup-fixer?_pos=1&amp;_sid=429166645&amp;_ss=r" tabindex="-1"><div class="product--labels" data-item="overline" data-text-color="body"><div class="product--label" data-background-color="accent-2" data-label-shape="round" data-label-type="sale">15% Off</div></div><img alt="Makeup Fixer BLP Beauty" class="product-card--image" height="2000" sizes="(max-width: 767px) calc(100vw - 32px), calc(25vw - 48px)" src="//blpbeauty.com/cdn/shop/products/MAKEUPFIXERPNG.png?v=1650509445&amp;width=1600" srcset="//blpbeauty.com/cdn/shop/products/MAKEUPFIXERPNG.png?v=1650509445&amp;width=200 200w, //blpbeauty.com/cdn/shop/products/MAKEUPFIXERPNG.png?v=1650509445&amp;width=300 300w, //blpbeauty.com/cdn/shop/products/MAKEUPFIXERPNG.png?v

In [ ]:

# Origin before job for BLP Beauty
product_data = []

product_cards = results[0].find_all('div', {'class':'product-card--details'})

if product_cards:
    for card in product_cards:
        
        # Product Name
        product_name_tag = card.find('p', {'class': 'product-card--title'})
        if product_name_tag:
            product_name = product_name_tag.get_text(strip=True)
            product_name = product_name.capitalize()
            print(f'Product Name: {product_name}')
        
        # Brand Name
        brand_name_tag = "BLP Beauty"
        print(f'Brand Name: {brand_name_tag}')

        # Sell Price (after discount)
        sell_price_tag = card.find('span', {'class': 'money'})
        if sell_price_tag:
            sell_price_tag = sell_price_tag.get_text(strip=True)
            # Remove Rp, dots, spaces
            cleaned = re.sub(r'[^\d]', '', sell_price_tag)  # Keeps only digits
            sell_price = int(cleaned) if cleaned.isdigit() else None
            print(f'Price: {sell_price}')
        else:
            sell_price = None
        
        # Extract Reviews Count
        reviews_count_tag = card.find('div', class_='product-rating--root')
        if reviews_count_tag and reviews_count_tag.get('data-count'):
            count = reviews_count_tag['data-count']
            print(f'Review count: {count}')
        else:
            print(f'Review count: none')

        # Extract rating
        rating_tag = card.find('div', class_='product-rating--root')
        
        if rating_tag and rating_tag.get('aria-label'):
            aria_label = rating_tag['aria-label']  # e.g. "5.0 out of 5.0 stars"
            match = re.search(r'(\d+\.?\d*) out of (\d+\.?\d*) stars', aria_label)
            if match:
                current_rating = float(match.group(1))
                max_rating = float(match.group(2))
                rating_decimal = round((current_rating / max_rating) * 100)
                print(f"Rating: {rating_decimal}")
            else:
                rating_decimal = None
        else:
            rating_decimal = None
            print("No rating found")

Product Name: Makeup fixer
Brand Name: BLP Beauty
Price: 109650
Review count: 10
Rating: 100


In [ ]:
# Translated into function and refactored

def get_blp_beauty_data(ti, page_no, skincare_product):
    if page_no > 1:
        page_num = 1
        while page_num <= page_no:
            url = f'https://blpbeauty.com/id/search?page={page_num}&q={skincare_product}&type=product#search_root'
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
            
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                results = soup.find_all('div', {'class': 'product-card--details'})
            except Exception as e:
                raise Exception(f"Failed to fetch or parse page: {e}")

            product_data = []

            for container in results:
                    # Product
                    product_name_tag = container.find('p', {'class': 'product-card--title'})
                    if product_name_tag:
                        product_name = product_name_tag.get_text(strip=True)
                        product_name = product_name.capitalize()

                    # Brand Name
                    brand_name_tag = "BLP Beauty"

                    # Price
                    sell_price_tag = container.find('span', {'class': 'money'})
                    if sell_price_tag:
                        sell_price_tag = sell_price_tag.get_text(strip=True)
                        # Remove Rp, dots, spaces
                        cleaned = re.sub(r'[^\d]', '', sell_price_tag)  # Keeps only digits
                        sell_price = int(cleaned) if cleaned.isdigit() else None
                    else:
                        sell_price = None

                    # Reviews count
                    reviews_count_tag = container.find('div', class_='product-rating--root')
                    if reviews_count_tag and reviews_count_tag.get('data-count'):
                        reviews_count = reviews_count_tag['data-count']

                    # Extract rating percentage from style attribute
                    rating_tag = card.find('div', class_='product-rating--root')
                    
                    if rating_tag and rating_tag.get('aria-label'):
                        aria_label = rating_tag['aria-label']  # e.g. "5.0 out of 5.0 stars"
                        match = re.search(r'(\d+\.?\d*) out of (\d+\.?\d*) stars', aria_label)
                        if match:
                            current_rating = float(match.group(1))
                            max_rating = float(match.group(2))
                            rating_percentage = round((current_rating / max_rating) * 100)
                        else:
                            rating_percentage = None
                    else:
                        rating_percentage = None

                    product_data.append({
                        'brand': product_brand,
                        'product_name': product_name,
                        'price': sell_price,
                        'reviews_count': reviews_count,
                        'rating_percentage': rating_percentage
                    })

            df = pd.DataFrame(product_data)
            ti.xcom_push(key='skincare_data', value=df.to_dict('records'))

            page_num += 1

            # Politeness Delay
            time.sleep(1)
    
    else:
        url = f'https://blpbeauty.com/id/search?page=1&q={skincare_product}&type=product#search_root'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            results = soup.find_all('div', {'class': 'product-card--details'})
        except Exception as e:
            raise Exception(f"Failed to fetch or parse page: {e}")

        product_data = []

        for container in results:
                # Product
                product_name_tag = container.find('p', {'class': 'product-card--title'})
                if product_name_tag:
                    product_name = product_name_tag.get_text(strip=True)
                    product_name = product_name.capitalize()

                # Brand Name
                brand_name = "BLP Beauty"

                # Price
                sell_price_tag = container.find('span', {'class': 'money'})
                if sell_price_tag:
                    sell_price_tag = sell_price_tag.get_text(strip=True)
                    # Remove Rp, dots, spaces
                    cleaned = re.sub(r'[^\d]', '', sell_price_tag)  # Keeps only digits
                    sell_price = int(cleaned) if cleaned.isdigit() else None
                else:
                    sell_price = None

                # Reviews count
                reviews_count_tag = container.find('div', class_='product-rating--root')
                if reviews_count_tag and reviews_count_tag.get('data-count'):
                    reviews_count = reviews_count_tag['data-count']

                # Extract rating percentage from style attribute
                rating_tag = card.find('div', class_='product-rating--root')
                
                if rating_tag and rating_tag.get('aria-label'):
                    aria_label = rating_tag['aria-label']  # e.g. "5.0 out of 5.0 stars"
                    match = re.search(r'(\d+\.?\d*) out of (\d+\.?\d*) stars', aria_label)
                    if match:
                        current_rating = float(match.group(1))
                        max_rating = float(match.group(2))
                        rating_percentage = round((current_rating / max_rating) * 100)
                    else:
                        rating_percentage = None
                else:
                    rating_percentage = None

                product_data.append({
                    'brand': brand_name,
                    'product_name': product_name,
                    'price': sell_price,
                    'reviews_count': reviews_count,
                    'rating_percentage': rating_percentage
                })

        df = pd.DataFrame(product_data)
        ti.xcom_push(key='skincare_data', value=df.to_dict('records')) # need to modify the key nto different table?

In [ ]:
product_data = []

product_cards = results[0].find_all('div', {'class':'product-card--details'})

# Extract data-product-brand and data-product-name
if product_cards:
    for card in product_cards:
        product_name = card.get('product-card--title')
        product_brand = card.get('data-product-brand')
        # Extract sell price
        sell_price = card.find('span', class_='sell-price').get_text(strip=True) if card.find('span', class_='sell-price') else 'N/A'
        
        # Extract reviews count
        reviews_count = card.find('span', class_='reviews-count').get_text(strip=True) if card.find('span', class_='reviews-count') else 'N/A'
        
        # Extract rating
        rating = card.find('div', class_='stars')['style'] if card.find('div', class_='stars') else 'N/A'
        highlighted_percentage = 'N/A'
        
        if rating != 'N/A':
            # Use regex to find the highlighted percentage value
            match = re.search(r'--highlighted-percentage:(\d+%)', rating)
            if match:
                highlighted_percentage = match.group(1)  # Get the percentage value
        
        print(f'Product Name: {product_name}, Brand: {product_brand}, Sell Price: {sell_price}, Reviews Count: {reviews_count}, Highlighted Percentage: {highlighted_percentage}')
else:
    print('No product card found.')

SyntaxError: invalid syntax (3762673937.py, line 7)

### Final Ecommerce scrape

In [6]:
# big ecommerce HTTP Requests
web_skincare = 'https://www.sociolla.com/search?q=moisturizer' #req can
response = requests.get(web_skincare)
response.status_code

200

In [5]:
# Soup Object
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>
<html lang="id"><head><meta charset="utf-8"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><title>Situs Kosmetik Terlengkap &amp; Terpercaya #1 di Indonesia - Sociolla</title><meta content="Situs Kosmetik Terlengkap &amp; Terpercaya #1 di Indonesia - Sociolla" name="title"/><meta content="Nikmati promo belanja hingga 50% Off, Extra Voucher, Bebas Ongkir Tanpa Syarat se-Indonesia! Temukan pilihan produk kecantikan 100% BPOM untuk setiap kebutuhanmu" name="description"/><meta content="online,store,terpercaya,beauty,kecantikan,skincare,kosmetik,makeup,beauty tools,catokan,kosmetik asli,skincare asli,toko online" id="meta-keywords" name="keywords"/><meta content="width=device-width,initial-scale=1,maximum-scale=1" name="viewport"/><meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-control"/><meta content="no-cache" http-equiv="Pragma"/><meta content="0" http-equiv="expires"/><link href="//soco-api.sociolla.com" rel="preconnect"/><link hre

In [4]:
# Results
results = soup.find_all('div', {'class': 'product-card--root'})
# len(results)
results

[]

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

# Apply patch for nested event loops
nest_asyncio.apply()

async def scrape_sociolla():
    async with async_playwright() as p:
        # ✅ Set user agent during browser launch
        browser = await p.chromium.launch(
            headless=True,
            args=[
                '--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
            ]
        )
        page = await browser.new_page()

        # Optional: Also set extra headers
        await page.set_extra_http_headers({
            "Accept-Language": "en-US,en;q=0.9,id;q=0.8",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Referer": "https://www.google.com",
            "Sec-Ch-Ua": '"Not/A)Brand";v="99", "Google Chrome";v="128", "Chromium";v="128"',
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": '"macOS"',
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Site": "same-origin",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-User": "?1",
            "Sec-Fetch-Dest": "document",
        })

        print("Going to Sociolla...")
        await page.goto("https://www.sociolla.com/search?q=moisturizer", timeout=60000)

        print("Waiting for products...")
        try:
            await page.wait_for_selector('.product__info', state='attached', timeout=30000)
        except Exception as e:
            print("Timeout:", str(e))
            content = await page.content()
            print("Page snippet:", content[:2000])
            await browser.close()
            return []

        print("Products found! Extracting data...")

        products = await page.eval_on_selector_all('.product__info', """
            elements => elements.map(el => {
                const brandEl = el.querySelector('a.product__brand');
                const nameEl = el.querySelector('a.product__name');
                const priceEl = el.querySelector('.product__price');
                const ratingEl = el.querySelector('.product__stars');
                const reviewsEl = el.querySelector('.product__reviews');
                const link = el.querySelector('a.product__name');

                return {
                    brand: brandEl ? brandEl.innerText.trim() : null,
                    name: nameEl ? nameEl.innerText.trim() : null,
                    price: priceEl ? priceEl.innerText.trim() : null,
                    rating: ratingEl ? ratingEl.innerText.trim() : null,
                    reviews: reviewsEl ? reviewsEl.innerText.trim() : null,
                    url: link ? 'https://www.sociolla.com' + link.getAttribute('href') : null
                };
            })
        """)

        await browser.close()
        return products

# Run it
results = asyncio.run(scrape_sociolla())

for product in results:
    print(f"Brand: {product['brand']}")
    print(f"Name: {product['name']}")
    print(f"Price: {product['price']}")
    print(f"Rating: {product['rating']} {product['reviews']}")
    print(f"URL: {product['url']}")
    print("-" * 50)

Going to Sociolla...
Waiting for products...
Products found! Extracting data...
Brand: Haple
Name: Sky Rose Moisturizer
Price: Rp119.000 Rp110.670 -7%
Rating: None None
URL: https://www.sociolla.com/face-cream-lotion/16544-sky-rose-moisturizer
--------------------------------------------------
Brand: The Originote
Name: Hyalucera Moisturizer
Price: Rp60.375
Rating: None None
URL: https://www.sociolla.com/face-gel/73207-hyalucera-moisturizer
--------------------------------------------------
Brand: Embryolisse
Name: Mattifying Moisturizer
Price: Rp389.000 Rp291.750 -25%
Rating: None None
URL: https://www.sociolla.com/face-cream-lotion/91887-mattifying-moisturizer
--------------------------------------------------
Brand: Tulus
Name: Probio Gel Moisturizer
Price: Rp229.000
Rating: None None
URL: https://www.sociolla.com/face-cream-lotion/85485-probio-gel-moisturizer
--------------------------------------------------
Brand: Emina
Name: Natura Pal Gel Moisturizer
Price: Rp42.500
Rating: Non

In [ ]:
#Refactor into function

import asyncio
import nest_asyncio
from playwright.async_api import async_playwright


async def get_sociolla_data():
    async with async_playwright() as p:
        # Apply patch for nested event loops
        nest_asyncio.apply()  

        # ✅ Set user agent during browser launch
        browser = await p.chromium.launch(
            headless=True,
            args=[
                '--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
            ]
        )
        page = await browser.new_page()

        # Optional: Also set extra headers
        await page.set_extra_http_headers({
            "Accept-Language": "en-US,en;q=0.9,id;q=0.8",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Referer": "https://www.google.com",
            "Sec-Ch-Ua": '"Not/A)Brand";v="99", "Google Chrome";v="128", "Chromium";v="128"',
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": '"macOS"',
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Site": "same-origin",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-User": "?1",
            "Sec-Fetch-Dest": "document",
        })

        print("Going to Sociolla...")
        await page.goto("https://www.sociolla.com/search?q=moisturizer", timeout=60000)

        print("Waiting for products...")
        try:
            await page.wait_for_selector('.product__info', state='attached', timeout=30000)
        except Exception as e:
            print("Timeout:", str(e))
            content = await page.content()
            print("Page snippet:", content[:2000])
            await browser.close()
            return []

        print("Products found! Extracting data...")

        products = await page.eval_on_selector_all('.product__info', """
            elements => elements.map(el => {
                const brandEl = el.querySelector('a.product__brand');
                const nameEl = el.querySelector('a.product__name');
                const priceEl = el.querySelector('.product__price');
                const ratingEl = el.querySelector('.product__stars');
                const reviewsEl = el.querySelector('.product__reviews');
                const link = el.querySelector('a.product__name');

                return {
                    brand: brandEl ? brandEl.innerText.trim() : null,
                    name: nameEl ? nameEl.innerText.trim() : null,
                    price: priceEl ? priceEl.innerText.trim() : null,
                    rating: ratingEl ? ratingEl.innerText.trim() : null,
                    reviews: reviewsEl ? reviewsEl.innerText.trim() : null,
                    url: link ? 'https://www.sociolla.com' + link.getAttribute('href') : null
                };
            })
        """)

        await browser.close()
        return products

def extract_sociolla_data(results):
    # Run it
    results = asyncio.run(get_sociolla_data())

    product_data = []


    for product in results:
        product_data.append({
            'brand': product['brand'],
            'product_name': product['name'],
            'price': product['price'],
            'reviews_count': product['rating'],
            'rating_percentage': product['reviews'] # still need to fix
        })

### Notes

In [ ]:
# Define the string with special characters
text = "Hello, World! This is a test string with special characters: #, @, $, %..."

# Define the regex pattern to match any character that is not alphanumeric or space
pattern = r'[^a-zA-Z0-9 ]'

# Use re.sub() to replace all occurrences of the pattern with an empty string
cleaned_text = re.sub(pattern, '', text)